<a href="https://colab.research.google.com/github/phi1z/1yanagiLab/blob/main/Brownian_Relaxation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 二次粒度分布からBrowm緩和時間を計算するWebUIです

## 利用方法

1.   <font color= “blue”>**「前提ファイルの読み込み」**</font>を実行する
2.   ディレクトリー名(`output_dir`)を入力して実行。左のフォルダーにdataなどが作成されます(**左のフォルダーアイコン**をクリックすると、エクスプローラーが開きます。フォルダーが見当たらないときは、上の<font color= “red”>**更新ボタン**</font>を押して下さい)
3.   data/[ディレクトリー名]/に**分布データ**(txtファイル)をアップロード。**「ファイルを確認」**を実行して**ファイルを選択**します

4.   **「分布を確認」**を押して、分布の範囲を**桁数で**指定
5.   <font color= “blue”>**「Brown緩和の算出」**</font>で、温度(単位は℃)(`Temperature`)と粘度(単位はcP)(`Viscosity`)を設定
6.   **「Brown緩和の算出」**を実行して、Brown緩和時間の平均を計算。また、自動で`out_put`に保存されます

###出力結果

===Condition===

Temperature: 310.15 K  **←絶対温度**

Viscosity: 0.6918 cp  **←液体の粘度**

===Result===

Average of Brownian relaxation time: 25.4744 s  **←平均Brown緩和時間**

===Distribution===


|Diameter| PDF| Relaxation time|
| ---- | ---- |---- |
|1.00000e+02| 0.00000e+00| 253892e-04|
|1.15000e+02| 0.00000e+00| 3.86138e-04|
|1.31000e+02| 0.00000e+00| 5.70773e-04|

**Diameter** ... 粒径

**PDF** ... 確率密度分布

**Diameter** ... Brown緩和時間

In [ ]:
#@title ##前提ファイルの読み込み (**とりあえず押して！**)

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import ipywidgets as widgets
from scipy.stats import lognorm
from scipy.optimize import curve_fit
from scipy import integrate
import os
from scipy.interpolate import interp1d
from tqdm import tqdm

!wget https://raw.githubusercontent.com/phi1z/1yanagiLab/main/data/T-eta.csv

T_eta_water = pd.read_csv("T-eta.csv")

def lognormal_pdf(x, mu, sigma):
    const = 1/(x * sigma * np.sqrt(2 * np.pi))
    likelihood = np.exp(-(np.log(x) - mu)**2 / (2 * sigma**2))
    return const * likelihood

def lognorm_dist(x, mu, sigma, scale1):
    return scale1 * lognormal_pdf(x, sigma=sigma, mu=mu)


def read_data(file_name, Pandas=False, unit="mu"):
    x_values = []
    y_values = []
    with open(file_name, 'r') as file: # 読み込むファイルを指定
        lines = file.readlines()
        for line in lines:
            split_line = line.split(',')
            x_values.append( float(split_line[0]) )
            y_values.append( float(split_line[1]) )

    if unit == "nano":
        x_values = np.array(x_values) * 1000
    if unit == "a":
        x_values = np.array(x_values) * 10000

    if Pandas:
        df = pd.DataFrame({'diameter': x_values, 'prob': y_values})
        return df
    else:
        return x_values, y_values

# Use nano meter as unit so that 10 nm = 1

def brownian_relaxtion_time(V_H,T, eta = 0.6918e-12):
    # V_H: Hydrodynamic volume
    # T: Temperature
    # eta: viscosity
    # k_B: Boltzmann constant
    # tau: relaxation time
    k_B = 1.38e-5
    tau = 3 * eta * V_H / (k_B*T)
    return tau

def D3_average_data(Diameters, pdf):
    D_cubic_ave = integrate.simpson(y=Diameters**3 * pdf, x=Diameters)
    All = integrate.simpson(y=pdf, x=Diameters)
    return D_cubic_ave / All

def make_ideal_lognorm(mu, sigma, range = 3, num = 1000, Pandas=False):
    X = np.logspace( (mu - range * sigma)*np.log10(np.e) , (mu + range * sigma)*np.log10(np.e), num )
    Y = lognormal_pdf(X, mu, sigma)
    if Pandas:
        df = pd.DataFrame({'x': X, 'y': Y})
        return df
    else:
        return X, Y

def D3_average_data(Diameters, pdf):
    D_cubic_ave = integrate.simpson(y=Diameters**3 * pdf, x=Diameters)
    All = integrate.simpson(y=pdf, x=Diameters)
    return D_cubic_ave / All

def get_c0_pdf(x, y, n, m, slide=0.01, log=False, nbin=100):
  # 線形補間を使用して連続的な関数を生成
  interpolation_function = interp1d(x, y, kind='linear')
  if log:
    series_X = np.logspace(np.log10(min(x)+slide), np.log10(max(x)-slide), n*m)
  else:
    series_X = np.linspace(min(x)+slide, max(x)-slide, n*m)
  series_Y = interpolation_function(series_X)
  pdf = pd.DataFrame({'diameter': series_X, 'pdf': series_Y})
  pdf['pdf'] = np.where(pdf['pdf'] <= 0, 1e-18, pdf['pdf'])
  print(f"Sampling diameters {m*n:,} times ...")
  random_Y = pdf['diameter'].sample(n=n*m, weights= pdf['pdf'])
  print("Sampling done!!!")
  cut_random_Y = np.array(random_Y).reshape(-1, m)
  prod_list = []
  print("Calculating product ...")
  for i in tqdm(range(len(cut_random_Y))):
    prod_list.append(np.prod(cut_random_Y[i]))
  print("Calculating done!!!")
  bin_range = np.logspace(np.log10(min(prod_list)), np.log10(max(prod_list)), nbin)
  n, bins, patches =  plt.hist(prod_list, bins=bin_range)
  plt.xscale('log')
  plt.show()
  df = pd.DataFrame({'diameter': bins[:-1], 'frequency': n})
  df['pdf'] = df['frequency'] / integrate.simpson(y=df['frequency'], x=df['diameter'])
  return df

def set_volume(df):
  df['volume'] = 1/6 * np.pi * (df['diameter']**3)
  df['pdf_d'] = df['prob'] / integrate.simpson(y=df['prob'], x=df['diameter'])
  df['pdf_v'] = df['prob'] / integrate.simpson(y=df['prob'], x=df['volume'])
  return df

os.makedirs('data/', exist_ok=True)

print("すべての前提ファイルが読み込まれました")

In [2]:
#@title ディレクトリー名の決定 { display-mode: "form"}
#@markdown ##実行ボタンを押してディレクトリー名を決定


#@markdown ##ディレクトリー名を入力
dir_name = "sample" # @param {"type":"string"}

os.makedirs("data", exist_ok=True)
dir_name_ud = os.path.join("data", dir_name)
os.makedirs(dir_name_ud, exist_ok=True)

print(f"ディレクトリ名: {dir_name}")

ディレクトリ名: sample



##ファイルをアップロード

###`data/[dir_name]/`に分布データ(.txt)を<font color= “orange”>**アップロード**</font>して下さい



In [ ]:
#@title ファイルを確認 { display-mode: "form"}
#@markdown ##1. 実行ボタンを押してファイルを確認
#@markdown ##2. 解析するファイルを選択 (実行ボタンは押さない)
files = os.listdir(dir_name_ud)
files = np.sort(files)

print(f"ファイル数: {len(files)}")

# ドロップダウンリストを作成
this_file = widgets.Dropdown(
    options=files,
    value=files[0],
    description='ファイルを選択:',
    disabled=False,
    layout={'width': '50%', 'height': '40px'},
    style={'description_width': 'initial'},
)

# 表示
display(this_file)

In [ ]:
# @title 分布の確認 { display-mode: "form"}

# @markdown ###分布の範囲を制限する(**指数で** 例:$10^4\to 4$)
min_diameter = 2 # @param {"type":"number"}
max_diameter = 5 # @param {"type":"number"}
Diameter_range = [10**min_diameter, 10**max_diameter]

df = read_data(os.path.join(dir_name_ud, this_file.value), Pandas=True, unit="nano")
df = df.loc[(df['diameter'] >= Diameter_range[0]) & (df['diameter'] <= Diameter_range[1])]

df['pdf'] = df['prob'] / integrate.simpson(y=df['prob'], x=df['diameter'])

cdf = integrate.cumulative_trapezoid(y=df['pdf'], x=df['diameter'], initial=0)

fig, ax1 = plt.subplots()

ax1.plot(df['diameter'], df['pdf'], label='Probability', linewidth=2.5)
ax1.set_xlabel('Diameter (nm)', fontsize=16)
ax1.set_ylabel('Probability Density', fontsize=16)
ax1.set_ylim(bottom=0)
plt.xscale('log')
plt.title(this_file.value.replace(".txt", ""), fontsize=18)

ax2 = ax1.twinx()
ax2.plot(df['diameter'], cdf, label='Cumulative', color='red', linewidth=2.5)
ax2.set_ylabel('Cumulative Distribution', fontsize=16)
ax2.set_ylim(bottom=0)

lines, labels = ax1.get_legend_handles_labels()
lines2, labels2 = ax2.get_legend_handles_labels()
plt.legend(lines + lines2, labels + labels2, loc='best')

plt.show()

print(f"Average: {integrate.simpson(y=df['pdf']*df['diameter'], x=df['diameter']):,.1f} nm")
print(f"Median:  {df.loc[(np.abs(cdf - 0.5)).argmin()]['diameter']:,.0f} nm")
print(f"Mode:    {df.loc[df['pdf'].idxmax()]['diameter']:,.0f} nm")

In [ ]:
#  @title Brown緩和の算出

os.makedirs("out_put", exist_ok=True)
os.makedirs(os.path.join("out_put", dir_name), exist_ok=True)

T_eta_func = interp1d(T_eta_water["Temp (dC)"], T_eta_water["eta (cP)"], kind="linear")

# @markdown ### 温度を入力 (摂氏)
Temperature = 37 # @param {"type":"number"}
# @markdown ### 粘度を入力 (単位は$\small\rm cP = mP\cdot s^{-1}$)
Viscosity = 0.6918 # @param {"type":"number"}
# @markdown ### もし、溶媒が水ならチェック(自動的に粘度が計算されます)
Water = True # @param {"type":"boolean"}

abs_Temp = Temperature + 273.15

if Water:
  Viscosity = T_eta_func(Temperature)

eta_nPs = Viscosity / 1000000000000


df = set_volume(df)
BRs = brownian_relaxtion_time(V_H=df['volume'], T=abs_Temp, eta=eta_nPs )
ave_BF = integrate.simpson(y=df['pdf_d']*BRs, x=df['diameter'])

output_text = "===Condition===\n"
output_text += f"Temperature: {abs_Temp} K\n"
output_text += f"Viscosity: {Viscosity:.4e} cp\n\n"

output_text += "===Result===\n"
output_text += f"Average of Brownian relaxation time: {ave_BF:.4e} s\n\n"

print(output_text)

plt.plot(BRs, df['pdf_d'])
plt.xlabel('Brownian relaxation time (s)')
plt.ylabel('Probability Density')
plt.title('Distribution of Brownian relaxation time')
plt.xscale('log')
plt.show()

output_text += "===Distribution===\n\n"
output_text += "Diameter, PDF, Relaxation time\n"
for index, row in df.iterrows():
  output_text += f"{row['diameter']:.5e}, {row['pdf_d']:.5e}, {BRs[index]:.5e}\n"

with open(os.path.join("out_put", dir_name, this_file.value.replace(".txt", "") + "_result.txt"), "w") as f:
  f.write(output_text)